In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
from flax import linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import values_vle_sle_sve_visc_model as values_mie_solid

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

# load pickle module
import pickle

In [2]:
folder_to_save = 'results'
os.makedirs(folder_to_save, exist_ok=True)

In [3]:
###########

filename = '../../computed_files/phase_equilibria_solid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
df_sle = pd.read_excel(excel_file, sheet_name='sle')
df_sve = pd.read_excel(excel_file, sheet_name='sve')

# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# triple point information interpolation
input_triple_interp = df_info['alpha'].to_numpy()
output_triple_interp = df_info[['rhovad_triple', 'rholad_triple', 'rhosad_triple',
                                'T_triple', 'P_triple']].to_numpy()
triple_interp = interp1d(input_triple_interp, output_triple_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

# Interpolating SLE
input_sle_interp = df_sle[['alpha', 'T_sle_model']].to_numpy()
output_sle_interp = df_sle[['P_sle_model', 'rhol_sle_model', 'rhos_sle_model']].to_numpy()
sle_interp = NearestNDInterpolator(input_sle_interp, output_sle_interp)

# Interpolating SVE
input_sve_interp = df_sve[['alpha', 'T_sve_model']].to_numpy()
output_sve_interp = df_sve[['P_sve_model', 'rhov_sve_model', 'rhos_sve_model']].to_numpy()
sve_interp = NearestNDInterpolator(input_sve_interp, output_sve_interp)

alphas_sle = np.unique(df_sle['alpha'])
Tsle_max = np.zeros_like(alphas_sle)
for i, alpha in enumerate(alphas_sle):
    Tsle_max[i] = df_sle[df_sle['alpha'] == alpha]['T_sle_model'].max()
sle_maxT_interp = interp1d(alphas_sle, Tsle_max, fill_value='extrapolate')

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp, 
                'sle_interp': sle_interp, 'sve_interp': sve_interp, 
                'triple_interp': triple_interp, 'sle_maxT_interp': sle_maxT_interp}

In [4]:
#########################
# Loading FE-ANN(s) EoS #
#########################

ckpt_folder = '../../ann_models/feanns_eos'
prefix_params = 'FE-ANN-EoS-params_'
###
Tscale = 'Tinv'
seed = 17
factor = 0.01
EPOCHS = 50000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params, step=EPOCHS)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}

fun_dic_solid = helper_jitted_funs(helmholtz_model, helmholtz_params)

########################### 
# Self-diffusivity models #
###########################

folder_diff = '../../ann_models/selfdiff_models'
hidden_layers = 2
neurons = 30
prefix = 'rhodiff-rho-Tinv-penalty'
ckpt_dir = folder_diff
seed = 1
features = hidden_layers * [neurons]
activation = 'softplus'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_rhodiff = {'params': state_restored['params']}
rhodiff_model = TransportModel_PVT_Tinv(features=features, output_activation=nn.softplus)

rhodiff_model_jit = jax.jit(lambda alpha, rhoad, Tad: rhodiff_model.apply(params_rhodiff, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
diff_fun = lambda alpha, rhoad, Tad: rhodiff_model_jit(alpha, rhoad, Tad)/rhoad

#########################
# Shear visocsity model #
#########################

folder_visc = '../../ann_models/visc_models'
hidden_layers = 2
neurons = 30
prefix = 'logvisc-rho-Tinv-penalty'
ckpt_dir = folder_visc
seed = 0
features = hidden_layers * [neurons]
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features)

logvisc_model_jit = jax.jit(lambda alpha, rhoad, Tad: logvisc_model.apply(params_logvisc, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
visc_fun = lambda alpha, rhoad, Tad: jnp.exp(logvisc_model_jit(alpha, rhoad, Tad))


##############################
# Thermal conductivity model #
##############################

folder_tcond = '../../ann_models/tcond_models'
hidden_layers = 3
neurons = 30
prefix = 'logtcond-rho-Tinv-penalty'
ckpt_dir = folder_tcond
seed = 1
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logtcond = {'params': state_restored['params']}
logtcond_model = TransportModel_PVT_Tinv(features=features)

logtcond_model_jit = jax.jit(lambda alpha, rhoad, Tad: logtcond_model.apply(params_logtcond, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
tcond_fun = lambda alpha, rhoad, Tad: jnp.exp(logtcond_model_jit(alpha, rhoad, Tad))


In [5]:
# Reading data from NIST
fluid_name = r'Methane'
filename = 'methane.xlsx'
DataFile = pd.ExcelFile(filename)

non_vibrational_deg_freedom = 6. # 6 for non-linear molecules
def Cvid_by_R(T):
    # valid from 50 to 1500 K
    k1 = 33298.
    k2 = 79933.
    k3 = 2086.9
    k4 = 41602.
    k5 = 991.96

    Cp = k1 + k2 * ((k3/T) /np.sinh(k3/T))**2  # J / (kmol K) 
    Cp += k4 * ((k5/T) /np.cosh(k5/T))**2 # J / (kmol K)
    Cp /= 1000.  # J / (mol K)

    Cp_R = Cp / R # Adim
    Cv_R = Cp_R - 1. # Adim 
    # Cvib = Cv_R - non_vibrational_deg_freedom/2. # Adim
    return Cv_R

In [6]:
file_types = ['vle', 'vle_visc', 'vle_sle_sve']
dict_values = {}
property_list = ['Pvle', 'rhol_vle', 'Hvap_vle', 'visc_vle', 'Psle', 'Psve', 'Hsub_sve', 'thermal_conductivity_vle', 'speed_of_sound_vle']
# collecting the values

for file_type in file_types:
    file = f'optimized_mie_params_{file_type}.csv'
    df = pd.read_csv(file)
    df = pd.read_csv(file)
    df.sort_values('of', inplace=True, ignore_index=True)
    sigma, eps, lambda_r = df.loc[0, ['sigma_sol',	'epsilon_sol',	'lr_sol']].values
    inc = [sigma, eps, lambda_r]
    values = values_mie_solid(inc, DataFile, fun_dic_solid, visc_fun, interpd_dict)

    values['sigma'], values['epsilon'], values['lambda_r'] = sigma, eps, lambda_r

    # Properties at VLE needed to compute the speed of sound
    Mw = values['Mw'] # g/mol
    alpha = helper_get_alpha(lambda_r, lambda_a=6)
    n_vle = len(values['Tvle_exp'])
    alpha_vle = alpha * np.ones(n_vle)
    Tvle_ad = values['Tvle_exp'] * values['T_factor']
    rholad_vle_model = values['rhol_vle_model'] * values['rho_factor']
    cvlad_vle_model = fun_dic_solid['cv_residual_fun'](alpha_vle, rholad_vle_model, Tvle_ad)
    cplad_vle_model = fun_dic_solid['cp_residual_fun'](alpha_vle, rholad_vle_model, Tvle_ad)
    kappaTad_vle_model = fun_dic_solid['isothermal_compressibility_fun'](alpha_vle, rholad_vle_model, Tvle_ad)

    cv_factor = 1. / R  # J/mol -> adim
    kappaT_factor = 1 / values['pressure_factor'] # Pa^-1 -> adim

    cv_res_liq_vle = cvlad_vle_model / cv_factor
    cp_res_liq_vle = cplad_vle_model / cv_factor
    kappaT_liq_vle = kappaTad_vle_model / kappaT_factor
    rho_liq_vle = values['rhol_vle_model']
    Tvle = values['Tvle_exp']

    # Computing Speed of sound
    Cvid_R = Cvid_by_R(Tvle)
    Cpid_R = Cvid_R + 1
    cv_liq_vle = cv_res_liq_vle + Cvid_R * R
    cp_liq_vle = cp_res_liq_vle + Cpid_R * R
    w2 = 1. / (rho_liq_vle * kappaT_liq_vle * (cv_liq_vle/cp_liq_vle) * (Mw / 1000.))
    w = np.sqrt(w2)
    values['speed_of_sound_vle_model'] = w

    # self diffusivity
    diff_liq = diff_fun(alpha_vle, rholad_vle_model, Tvle_ad) / values['diff_factor']
    values['diff_vle_model'] = diff_liq
    # thermal conductivity
    tcond_liq = tcond_fun(alpha_vle, rholad_vle_model, Tvle_ad) / values['tcond_factor']
    Cvid_R_nonvib = Cvid_R - non_vibrational_deg_freedom/2
    tcond_liq_corrected = tcond_liq + rho_liq_vle * diff_liq * (Cvid_R_nonvib * R)
    values['thermal_conductivity_vle_ann'] = np.asarray(tcond_liq)
    values['thermal_conductivity_vle_vib'] = rho_liq_vle * diff_liq * (Cvid_R_nonvib * R)
    values['thermal_conductivity_vle_model'] = tcond_liq_corrected

    ######### computing MAPE values
    for property in property_list:
        values_model = values[f'{property}_model']
        values_nist = values[f'{property}_exp']
        if 'vle' in property:
            where = values['where_vle']
        elif 'sle' in property:
            where = values['where_sle']
        elif 'sve' in property:
            where = values['where_sve']
        else:
            where = np.ones_like(values_model, dtype=bool)
        mape = 100 * np.nanmean(np.abs(values_model/values_nist- 1), where=where)
        values[f'{property}_mape'] = mape

    #####################################################
    # Computing self-diffusivity against published data #
    #####################################################
    Mw = DataFile.parse('info')['Mw'][0]  # g/mol
    df_diff_lit = DataFile.parse('diff')
    df_diff_lit = df_diff_lit[df_diff_lit['Phase/State'].isin([0, 1, 2])].reset_index(drop=True)
    T_diff_lit = df_diff_lit['T (K)'].to_numpy()         # [K]
    rho_diff_lit = df_diff_lit['ρ (kg/m3)'].to_numpy() * (1000/Mw)   # [mol/m3]
    diff_lit = df_diff_lit['Dx109 (m2/s)'].to_numpy() * 1e-9   # [m2/s]
    phase_diff_lit = df_diff_lit['Phase/State'].to_numpy() # [adim]

    T_diff_lit_ad = T_diff_lit * values['T_factor']                    # [adim]
    rho_diff_lit_ad = rho_diff_lit * values['rho_factor']  # [adim]

    alpha_diff = alpha * np.ones_like(T_diff_lit_ad)
    diff_model = diff_fun(alpha_diff, rho_diff_lit_ad, T_diff_lit_ad) / values['diff_factor']  # [m2/s]

    mape_diff = 100 * np.nanmean(np.abs(diff_model/diff_lit - 1))

    # saving values
    values['T_diff_lit'] = T_diff_lit
    values['rho_diff_lit'] = rho_diff_lit
    values['diff_lit'] = diff_lit
    values['diff_model'] = diff_model
    values['diff_mape'] = mape_diff
    values['phase_diff'] = phase_diff_lit

    ######################################################

    # saving values
    dict_values[file_type] = values
    print(file_type, values['sigma'], values['epsilon'], values['lambda_r'])

# define dictionary

# create a binary pickle file 
pickle_to_save = os.path.join(folder_to_save, 'values_per_of.pkl')
f = open(pickle_to_save,"wb")

# write the python object (dict) to pickle file
pickle.dump(dict_values, f)

# close file
f.close()

# to load the saved object
# file = open("values_per_of.pkl",'rb')
# object_file = pickle.load(file)

2025-03-20 09:39:24.928347: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-20 09:39:25.008347: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-20 09:39:25.032343: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-20 09:39:25.040365: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-20 09:39:25.457714: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none
2025-03-20 09:39:25.705410: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:606] fastpath_data is none


vle 3.783505687459526 162.32029697807357 14.203692109352684
vle_visc 3.815393539930321 139.81080742682707 10.829608892138523
vle_sle_sve 3.7627778232112448 139.96367385481415 10.683948998914046


In [7]:
vle_properties = ['Pvle', 'rhol_vle', 'Hvap_vle']
sle_properties = ['Psle']
sve_properties = ['Psve', 'Hsub_sve']
property_list = ['Pvle', 'rhol_vle', 'Hvap_vle', 'Psle', 'Psve', 'Hsub_sve', 'visc_vle', 'thermal_conductivity_vle', 'speed_of_sound_vle']
color_cell = 'lightgray!40'
width_name = '2cm'
n_of = len(file_types)
objective_types = [r'$\textnormal{OF}_1$', r'$\textnormal{OF}_2$', r'$\textnormal{OF}_3$', r'$\textnormal{OF}_4$']


dict_properties_best = {}
for property in property_list:
    best_value = 1000.
    best_file_type = None

    for file_type in file_types:
        values = dict_values[file_type]
        mape = values[f"{property}_mape"]
        if mape < best_value:
            best_value = mape
            best_file_type = file_type
    dict_properties_best[property] = best_file_type


table_latex = """ """

table_latex += f'\multirow{{{n_of}}}{{{width_name}}}{{{fluid_name}}}'


for file_type, objective_type in zip(file_types, objective_types):
    values = dict_values[file_type]

    table_latex += f' & '
    table_latex += objective_type

    table_latex += f' & {values["sigma"]:.2f} & {values["epsilon"]:.2f} & {values["lambda_r"]:.2f}'

    for property in property_list:
        table_latex += f' & '
        if 'vle' in file_type and property in vle_properties:
            table_latex += f' \cellcolor{{{color_cell}}}'
        elif 'sle' in file_type and property in sle_properties:
            table_latex += f' \cellcolor{{{color_cell}}}'
        elif 'sve' in file_type and property in sve_properties:
            table_latex += f' \cellcolor{{{color_cell}}}'
        elif 'visc' in file_type and property == 'visc_vle':
            table_latex += f' \cellcolor{{{color_cell}}}'

        best_of_property = dict_properties_best[property]
        value_mape = values[f"{property}_mape"]
        if not np.isnan(value_mape):
            if best_of_property == file_type:
                table_latex += f' \\textbf{{{value_mape:.2f}}}'
            else:
                table_latex += f' {value_mape:.2f}'
        else:
            table_latex += f' -- '

    table_latex += f" \\\ \n  "

print(table_latex)

filename = "summary_table.md"
path_to_save = os.path.join(folder_to_save, filename)
text_file = open(path_to_save, "w")
text_file.write(table_latex)
text_file.close()

 \multirow{3}{2cm}{Methane} & $\textnormal{OF}_1$ & 3.78 & 162.32 & 14.20 &  \cellcolor{lightgray!40} \textbf{0.50} &  \cellcolor{lightgray!40} 3.35 &  \cellcolor{lightgray!40} \textbf{3.50} &  66.12 &  38.81 &  9.93 &  24.76 &  \textbf{3.16} &  \textbf{5.93} \\ 
   & $\textnormal{OF}_2$ & 3.82 & 139.81 & 10.83 &  \cellcolor{lightgray!40} 5.25 &  \cellcolor{lightgray!40} 7.48 &  \cellcolor{lightgray!40} 4.58 &  38.33 &  9.82 &  \textbf{0.64} &  \cellcolor{lightgray!40} \textbf{2.47} &  12.27 &  9.11 \\ 
   & $\textnormal{OF}_3$ & 3.76 & 139.96 & 10.68 &  \cellcolor{lightgray!40} 4.62 &  \cellcolor{lightgray!40} \textbf{3.29} &  \cellcolor{lightgray!40} 5.55 &  \cellcolor{lightgray!40} \textbf{35.94} &  \cellcolor{lightgray!40} \textbf{3.96} &  \cellcolor{lightgray!40} 0.89 &  6.92 &  8.31 &  10.04 \\ 
  
